# Быстрое обучение модели

Мы собрали данные по докторам, теперь мы можем обучить простую модель для предсказания стоимости приёма

In [1]:
import json
import numpy as np

Загрузим данные из JSON-a

In [2]:
with open('doctors.json', 'r') as f:
    doctors = json.load(f)

Проверим содержимое объекта

In [3]:
doctors[0]

{'experience': 23.0,
 'is_first_category': True,
 'is_phd': True,
 'name_parts': ['Сергейко', 'Анатолий', 'Анатольевич'],
 'price': 1700.0,
 'proffesions': ['Хирург', 'Эндоскопист']}

Посмотрим на различные специализации докторов, используя класс Counter для подсчёта статистики

In [4]:
from collections import Counter

In [5]:
prof_cnt = Counter()
for doctor in doctors:
    prof_cnt.update(doctor['proffesions'])
prof_cnt

Counter({'': 14,
         'Акушер': 361,
         'Аллерголог': 61,
         'Андролог': 118,
         'Анестезиолог': 41,
         'Артролог': 8,
         'Венеролог': 128,
         'Вертебролог': 44,
         'Врач ЛФК': 63,
         'Врач функциональной диагностики': 90,
         'Гастроэнтеролог': 135,
         'Гематолог': 13,
         'Генетик': 13,
         'Гепатолог': 28,
         'Гинеколог': 508,
         'Гинеколог-эндокринолог': 89,
         'Гирудотерапевт': 44,
         'Гомеопат': 22,
         'Дерматолог': 265,
         'Диабетолог': 4,
         'Диетолог': 47,
         'Иммунолог': 53,
         'Инфекционист': 16,
         'Кардиолог': 200,
         'Кардиохирург': 1,
         'Кинезиолог': 19,
         'Колопроктолог': 38,
         'Косметолог': 236,
         'Логопед': 20,
         'Лор (отоларинголог)': 213,
         'Маммолог': 72,
         'Мануальный терапевт': 151,
         'Массажист': 123,
         'Миколог': 18,
         'Нарколог': 72,
         'Невролог': 

In [6]:
prof_set = frozenset(prof_cnt.keys())
prof_set_list = list(prof_set)

In [29]:
with open('prof_set_list.json', 'w') as f:
    json.dump(prof_set_list, f)

In [7]:
len(prof_set)

84

Создадим выборку для обучения

Удобно в комментариях писать название признака

In [8]:
X, y = [], []

for doctor in doctors:
    features = []
    # 0 - experience
    features.append(doctor['experience'] or 0)
    # 1 - is_first_category
    features.append(float(doctor['is_first_category']))
    # 2 - is_phd
    features.append(float(doctor['is_phd']))
    # 3 - len(proffesions)
    features.append(len(doctor['proffesions']))
    # 4 - 87 one hot proffesions
    for p in prof_set_list:
        features.append(float(p in doctor['proffesions']))
        
    X.append(features)
    y.append(doctor['price'] or 0)
    
X = np.array(X)
y = np.array(y)

Обучим простые модели

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.dummy import DummyRegressor

DummyRegressor это простой бейзлайн - предсказание константой

In [10]:
np.sqrt(-cross_val_score(DummyRegressor(), X, y, cv=3, scoring='neg_mean_squared_error').mean())

2292.987742999036

In [11]:
np.sqrt(-cross_val_score(LinearRegression(), X, y, cv=3, scoring='neg_mean_squared_error').mean())

1019169977242.8812

In [12]:
np.sqrt(-cross_val_score(RandomForestRegressor(), X, y, cv=3, scoring='neg_mean_squared_error').mean())

2516.96455239046

In [13]:
np.sqrt(-cross_val_score(RandomForestRegressor(n_estimators=100), X, y, cv=3, scoring='neg_mean_squared_error').mean())

2465.242921355518

In [14]:
np.sqrt(-cross_val_score(KNeighborsRegressor(), X, y, cv=3, scoring='neg_mean_squared_error').mean())

2360.2875565348754

In [15]:
np.sqrt(-cross_val_score(KNeighborsRegressor(n_neighbors=10), X, y, cv=3, scoring='neg_mean_squared_error').mean())

2293.558275438247

In [16]:
import xgboost as xgb

In [17]:
np.sqrt(-cross_val_score(xgb.XGBRegressor(), X, y, cv=3, scoring='neg_mean_squared_error').mean())

2186.5772879904057

xgboost лучше всех, посмотрим на самые важность признаков

In [18]:
algo = xgb.XGBRegressor().fit(X, y)
sorted(zip(algo.feature_importances_, range(X.shape[1])), reverse=True)

[(0.23649634, 0),
 (0.14306569, 3),
 (0.045255475, 2),
 (0.035036497, 63),
 (0.035036497, 55),
 (0.030656934, 79),
 (0.030656934, 36),
 (0.02919708, 60),
 (0.027737226, 1),
 (0.023357663, 73),
 (0.02189781, 82),
 (0.02189781, 76),
 (0.018978102, 30),
 (0.017518248, 34),
 (0.016058395, 40),
 (0.01459854, 71),
 (0.01459854, 13),
 (0.013138686, 46),
 (0.011678832, 70),
 (0.011678832, 61),
 (0.011678832, 42),
 (0.011678832, 41),
 (0.011678832, 27),
 (0.010218978, 81),
 (0.010218978, 65),
 (0.010218978, 23),
 (0.010218978, 14),
 (0.008759124, 85),
 (0.008759124, 77),
 (0.008759124, 59),
 (0.008759124, 39),
 (0.008759124, 26),
 (0.00729927, 12),
 (0.00729927, 5),
 (0.005839416, 54),
 (0.005839416, 18),
 (0.005839416, 11),
 (0.004379562, 67),
 (0.004379562, 49),
 (0.004379562, 31),
 (0.004379562, 22),
 (0.004379562, 8),
 (0.004379562, 6),
 (0.002919708, 69),
 (0.002919708, 52),
 (0.002919708, 35),
 (0.002919708, 28),
 (0.001459854, 86),
 (0.001459854, 58),
 (0.001459854, 51),
 (0.001459854, 4

In [19]:
sorted(zip(algo.feature_importances_, range(X.shape[1])), reverse=True)[:7]

[(0.23649634, 0),
 (0.14306569, 3),
 (0.045255475, 2),
 (0.035036497, 63),
 (0.035036497, 55),
 (0.030656934, 79),
 (0.030656934, 36)]

0 - experience

3 - len(proffesions)

2 - is_phd

In [20]:
prof_set_list[87 - 4]

'Фониатр'

In [21]:
prof_set_list[19 - 4]

'Врач ЛФК'

In [22]:
prof_set_list[78 - 4]

'Флеболог'

In [23]:
prof_set_list[15 - 4]

'Нефролог'

Сохраним модель

In [24]:
algo.booster().save_model('xgboost_model')

Загрузим модель с диска и проверим, что всё корректно

In [25]:
bst = xgb.Booster()
bst.load_model('xgboost_model')

In [26]:
clf = xgb.XGBRegressor()
booster = xgb.Booster()
booster.load_model('xgboost_model')
clf._Booster = booster

In [27]:
clf.predict(X)

array([2006.3156, 2191.837 , 2861.2122, ..., 2339.8147,  565.0743,
       1137.5759], dtype=float32)

In [28]:
algo.predict(X)

array([2006.3156, 2191.837 , 2861.2122, ..., 2339.8147,  565.0743,
       1137.5759], dtype=float32)